In [41]:
import os
import pandas as pd
import numpy as np

In [42]:
file_list = os.listdir('data')
excel_list = []
for file_name in file_list:
    data = pd.ExcelFile('data/' + file_name)
    excel_list.append(data)


In [43]:
file_list

['국민건강보험공단_환경성질환(아토피) 의료이용정보_20231231.xlsx',
 '국민건강보험공단_환경성질환(비염) 의료이용정보_20231231.xlsx',
 '국민건강보험공단_환경성질환(천식) 의료이용정보_20231231.xlsx']

In [44]:
area_cd = excel_list[0].parse('시군구코드')

In [45]:
area_cd['지역명'] = area_cd['시도명'] + ' ' + area_cd['시군구명']

In [46]:
area_cd

,시군구코드,시도명,시군구명,지역명
0,11110,서울특별시,종로구,서울특별시 종로구
1,11140,서울특별시,중구,서울특별시 중구
2,11170,서울특별시,용산구,서울특별시 용산구
3,11200,서울특별시,성동구,서울특별시 성동구
4,11215,서울특별시,광진구,서울특별시 광진구
...,...,...,...,...
259,51790,강원특별자치도,화천군,강원특별자치도 화천군
260,51800,강원특별자치도,양구군,강원특별자치도 양구군
261,51810,강원특별자치도,인제군,강원특별자치도 인제군
262,51820,강원특별자치도,고성군,강원특별자치도 고성군


In [47]:
disease_list = ['아토피', '비염', '천식']

In [50]:
data_dict = {}
for i, disease in enumerate(disease_list):
    data1 = excel_list[i].parse('외래입원주부상병')
    data2 = excel_list[i].parse('입원주부상병')
    result = pd.concat([data1, data2], axis=0, ignore_index=True)
    data_dict[disease] = result

In [51]:
for disease in disease_list:
    data = data_dict[disease]
    data['요양개시연월'] = pd.to_datetime(data['요양개시연월'], format='%Y-%m')
    data = data[(data['요양개시연월'].dt.year >= 2020) & (data['요양개시연월'].dt.year <= 2022)]
    data = data.groupby(['요양개시연월', '주소(시군구)'], as_index=False)['진료에피소드 건수'].sum()
    data = data.rename(columns={'진료에피소드 건수': disease})
    data_dict[disease] = data

In [55]:
data_dict[disease_list[0]]

,요양개시연월,주소(시군구),아토피
0,2020-01-01,11110,390
1,2020-01-01,11140,307
2,2020-01-01,11170,650
3,2020-01-01,11200,836
4,2020-01-01,11215,946
...,...,...,...
8995,2022-12-01,51790,47
8996,2022-12-01,51800,30
8997,2022-12-01,51810,111
8998,2022-12-01,51820,82


In [57]:
df = data_dict[disease_list[0]]

for i in range(1, len(disease_list)):
    df = pd.merge(df, data_dict[disease_list[i]], on=['요양개시연월', '주소(시군구)'], how='outer')

In [58]:
df

,요양개시연월,주소(시군구),아토피,비염,천식
0,2020-01-01,11110,390,6624,494
1,2020-01-01,11140,307,5553,699
2,2020-01-01,11170,650,9548,827
3,2020-01-01,11200,836,13242,1439
4,2020-01-01,11215,946,15393,1641
...,...,...,...,...,...
8995,2022-12-01,51790,47,1267,113
8996,2022-12-01,51800,30,983,66
8997,2022-12-01,51810,111,1217,255
8998,2022-12-01,51820,82,1070,130


In [59]:
df_add_area = pd.merge(df, area_cd, left_on='주소(시군구)', right_on='시군구코드', how='left')

In [60]:
df_add_area.head()

,요양개시연월,주소(시군구),아토피,비염,천식,시군구코드,시도명,시군구명,지역명
0,2020-01-01,11110,390,6624,494,11110,서울특별시,종로구,서울특별시 종로구
1,2020-01-01,11140,307,5553,699,11140,서울특별시,중구,서울특별시 중구
2,2020-01-01,11170,650,9548,827,11170,서울특별시,용산구,서울특별시 용산구
3,2020-01-01,11200,836,13242,1439,11200,서울특별시,성동구,서울특별시 성동구
4,2020-01-01,11215,946,15393,1641,11215,서울특별시,광진구,서울특별시 광진구


In [61]:
df_add_area.to_excel('질병데이터_전처리.xlsx', index=False)